# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,novo aripuana,-5.1206,-60.3797,26.92,73,82,0.94,BR,1698317413
1,1,hermanus,-34.4187,19.2345,19.29,75,81,4.90,ZA,1698317355
2,2,bredasdorp,-34.5322,20.0403,26.89,38,94,3.99,ZA,1698317414
3,3,chibougamau,49.9168,-74.3659,1.79,100,100,2.06,CA,1698317415
4,4,weno,7.4515,151.8468,26.97,94,100,3.60,FM,1698317415


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_mz = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 15)]

#Wind speed less than 3.2
city_data_df_mz = city_data_df.loc[(city_data_df["Wind Speed"] < 3.2)]

#no clouds
city_data_df_mz = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df_mz = city_data_df.dropna()

# Display sample data
city_data_df_mz

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,novo aripuana,-5.1206,-60.3797,26.92,73,82,0.94,BR,1698317413
1,1,hermanus,-34.4187,19.2345,19.29,75,81,4.90,ZA,1698317355
2,2,bredasdorp,-34.5322,20.0403,26.89,38,94,3.99,ZA,1698317414
3,3,chibougamau,49.9168,-74.3659,1.79,100,100,2.06,CA,1698317415
4,4,weno,7.4515,151.8468,26.97,94,100,3.60,FM,1698317415
...,...,...,...,...,...,...,...,...,...,...
593,593,kununurra,-15.7667,128.7333,32.99,29,88,2.57,AU,1698317733
594,594,sarykemer,43.0000,71.5000,18.71,51,79,9.88,KZ,1698317734
595,595,cabedelo,-6.9811,-34.8339,27.37,69,54,5.22,BR,1698317735
596,596,berdigestyakh,62.0989,126.6992,-13.61,100,89,1.35,RU,1698317735


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[["City", "Country","Lat","Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(100)

C:\Users\michaelz\AppData\Local\Temp\ipykernel_556\392332629.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,novo aripuana,BR,-5.1206,-60.3797,73,
1,hermanus,ZA,-34.4187,19.2345,75,
2,bredasdorp,ZA,-34.5322,20.0403,38,
3,chibougamau,CA,49.9168,-74.3659,100,
4,weno,FM,7.4515,151.8468,94,
...,...,...,...,...,...,...
95,iqaluit,CA,63.7506,-68.5145,80,
96,smithers,CA,54.7804,-127.1743,79,
97,wailua homesteads,US,22.0669,-159.3780,79,
98,ola,RU,59.5833,151.2833,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
novo aripuana - nearest hotel: Hotel Novo Aripuanã
hermanus - nearest hotel: Aloe guest house
bredasdorp - nearest hotel: Victoria Hotel
chibougamau - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
bethel - nearest hotel: Hampton Inn Danbury
yerba buena - nearest hotel: howard johnson
mahina - nearest hotel: Motu Martin
waitangi - nearest hotel: Hotel Chathams
puerto natales - nearest hotel: Dorotea Patagonia Hostel
tofol - nearest hotel: Nautilus Resort
baker city - nearest hotel: Geiser Grand Hotel
koror - nearest hotel: West Plaza Desekel
albany - nearest hotel: No hotel found
tegueste - nearest hotel: Hotel Aguere
sabang - nearest hotel: Sabang Hill
forster - nearest hotel: No hotel found
stanley - nearest hotel: Hotel 52
adamstown - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
krasnyy chikoy - nearest hotel: Гостиница ЗАО "Слюдянка"
tabou - nearest hotel: hôtel le rochet
blackmans bay - nearest hotel: Villa Howden
r

nishinoomote - nearest hotel: ゼウスハウス
labrador city - nearest hotel: Carol Inn
xiaoweizhai - nearest hotel: No hotel found
badger - nearest hotel: Hotel North Pole
bikenibeu village - nearest hotel: Otintaai Hotel
hithadhoo - nearest hotel: Pebbles Inn
hyesan - nearest hotel: No hotel found
saldanha - nearest hotel: Hoedjesbaai
georgetown - nearest hotel: Page 63 hostel
nuuk - nearest hotel: Nordbo
mossel bay - nearest hotel: Protea Hotel Mossel Bay
zlocieniec - nearest hotel: Pałac w Cieszynie
luganville - nearest hotel: Beachfront Resort
keflavik - nearest hotel: Núpan Deluxe
korla - nearest hotel: Silver Star Hotel
vera cruz - nearest hotel: No hotel found
vera - nearest hotel: Hotel La Giralda
pionerskiy - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
matadi - nearest hotel: Hôtel Métroplole
valley - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
as salamiyah - nearest hotel: فندق السلام
qulsary - nearest hot

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.